In [1]:
import os

root = '../../'
os.chdir(root)

import glob

# import cv2
import numpy as np
from matplotlib import pyplot as plt

import random

#import png
import itertools

from skimage.io import imread,imsave
import shutil
import getopt
from multiprocessing import Pool 
import time
import sys
sys.path.append('caffe/python')
import caffe

In [2]:
dataset_root ='dataset/cuhk03/'
DATA_DIR = dataset_root+'data/'
multiply = 25
num_neg_base = 2
test_id_number = 486


M=160   #height
N=80   #width

num_pos = 1*multiply
num_neg = int(num_neg_base*multiply)
# how to get hard negative samples???!!!!
DIR = DATA_DIR

# 生成序列长度表

In [12]:
set_no = 1
fileNumList = dataset_root+'split/totalNumListDict.txt'
fp = open(fileNumList,'w')
phase = 'train'
filename_train = dataset_root+'exp_set/set%02d_%s_noval.txt'%((set_no),phase)

file_object = open(filename_train)

try:
    all_the_text = file_object.read( )
finally:
    file_object.close( )
lines = all_the_text.split('\n')

phase = 'test'
filename_train = dataset_root+'exp_set/set%02d_%s_noval.txt'%((set_no),phase)

file_object = open(filename_train)

try:
    all_the_text = file_object.read( )
finally:
    file_object.close( )
lines.extend(all_the_text.split('\n'))
lines.sort()
new_lines = []
for filename in lines:
    if filename!='' :
        campair_no = filename.split(',')[0]
        person_id = filename.split(',')[1]
        fileser = DATA_DIR+'campair_'+str(int(campair_no))+'/'+'%02d'%int(campair_no)+'_'+'%04d'%int(person_id)+'_'+'*.jpg'
        globList = glob.glob(fileser)
        filename = '%02d'%int(campair_no)+'_'+'%04d'%int(person_id)+' '+str(len(globList))+'\n'
        fp.write(filename)
        fp.flush()

# 生成sin_img列表

In [5]:
for set_idx in range(20):
    set_no = set_idx+1



    image_list=[]
    map_list=[]

    for conf_no in range(2):
        image_list=[]

        if conf_no==0:
            phase = 'train'
        else:
            phase = 'test'


        filename_train = dataset_root+'exp_set/set%02d_%s_noval.txt'%((set_no),phase)
        filename_trainlist = dataset_root+'split/sin_img/set%02d_%s_noval_pair.txt'%((set_no),phase)

        file_object = open(filename_train)
        fp = open(filename_trainlist,'w')
        try:
            all_the_text = file_object.read( )
        finally:
            file_object.close( )


        lines = all_the_text.split('\n')
        new_lines = []
        for filename in lines:
            if filename!='' :
                campair_no = int(filename.split(',')[0])
                person_id = int(filename.split(',')[1])
                #print int(campair_no)
                if campair_no<=3:                
                    new_lines.append(filename)
        random.shuffle(new_lines)
        lines=new_lines[0:1160]
        print len(lines)
        new_lines = []
        for filename in lines:
            if filename!='' :
                new_lines.append(filename)
        lines=new_lines
        

        for filename in lines:
            if filename!='' :
                campair_no = int(filename.split(',')[0])
                person_id = int(filename.split(',')[1])
                #print int(campair_no)
                for img_no in range(10):
                    this_filename = DATA_DIR +  'campair_%d/'%campair_no + '%02d_%04d_%02d.jpg'%(campair_no,person_id,img_no+1)
                    if os.path.isfile(this_filename):
                        image_list.append(this_filename)


        training_pairs = []
        for imagename in image_list:
            # for every image, find the corresponding view of image (positive)
            pair_group_name = imagename[-14:-12]
            identity_name = imagename[-11:-7]
            view_name = int(imagename[-6:-4])/5 
            for i in range(num_pos):
                imagename_pair = imagename[0:-6] + '%02d'%(random.randint(0,4)+1+(1-view_name)*5) + '.jpg'
                if os.path.isfile(imagename_pair):
                    pair_name = imagename + ' ' + imagename_pair + ' 1'
                    training_pairs.append(pair_name)
            # collect nagative training samples
            for i in range(num_neg):
                imagename_pair = image_list[random.randint(0,len(image_list))-1]
                identity_name_pair = imagename_pair[-11:-7]
                if identity_name_pair!=identity_name:
                    if os.path.isfile(imagename_pair):
                        pair_name = imagename + ' ' + imagename_pair + ' 0'
                        training_pairs.append(pair_name)


        random.shuffle(training_pairs)
       
        for line in training_pairs:
            fp.write(line+'\n')
            fp.flush()
        fp.close()
        print 'summary: set:%d - %s'%(set_no,phase)
        print 'person:%d'%(len(lines))
        print 'img num:%d'%len(image_list)
        print "tpairs_num:%d"%(len(training_pairs))
        

1160
summary: set:1 - train
person:1160
img num:11243
tpairs_num:807852
100
summary: set:1 - test
person:100
img num:965
tpairs_num:68838
1160
summary: set:2 - train
person:1160
img num:11232
tpairs_num:806675
100
summary: set:2 - test
person:100
img num:971
tpairs_num:69359
1160
summary: set:3 - train
person:1160
img num:11206
tpairs_num:804509
100
summary: set:3 - test
person:100
img num:975
tpairs_num:69639
1160
summary: set:4 - train
person:1160
img num:11234
tpairs_num:806983
100
summary: set:4 - test
person:100
img num:966
tpairs_num:68840
1160
summary: set:5 - train
person:1160
img num:11231
tpairs_num:806749
100
summary: set:5 - test
person:100
img num:965
tpairs_num:68733
1160
summary: set:6 - train
person:1160
img num:11209
tpairs_num:804669
100
summary: set:6 - test
person:100
img num:984
tpairs_num:70513
1160
summary: set:7 - train
person:1160
img num:11234
tpairs_num:806911
100
summary: set:7 - test
person:100
img num:959
tpairs_num:68197
1160
summary: set:8 - train
person

# 生成img_seq列表:
序列长度为3，序列位置图片的补集作为序列集

In [7]:
for set_idx in range(20):
    set_no = set_idx+1



    image_list=[]
    map_list=[]

    for conf_no in range(2):
        image_list=[]

        if conf_no==0:
            phase = 'train'
        else:
            phase = 'test'


        filename_train = dataset_root+'exp_set/set%02d_%s_noval.txt'%((set_no),phase)
        filename_trainlist = dataset_root+'split/img_seq/set%02d_%s_noval_pair.txt'%((set_no),phase)

        file_object = open(filename_train)
        fp = open(filename_trainlist,'w')
        try:
            all_the_text = file_object.read( )
        finally:
            file_object.close( )


        lines = all_the_text.split('\n')
        new_lines = []
        for filename in lines:
            if filename!='' :
                campair_no = int(filename.split(',')[0])
                person_id = int(filename.split(',')[1])
                #print int(campair_no)
                if campair_no<=3:                
                    new_lines.append(filename)
        random.shuffle(new_lines)
        lines=new_lines[0:1160]
        print len(lines)
        new_lines = []
        for filename in lines:
            if filename!='' :
                new_lines.append(filename)
        lines=new_lines
        

        for filename in lines:
            if filename!='' :
                campair_no = int(filename.split(',')[0])
                person_id = int(filename.split(',')[1])
                #print int(campair_no)
                for img_no in range(10):
                    this_filename = DATA_DIR +  'campair_%d/'%campair_no + '%02d_%04d_%02d.jpg'%(campair_no,person_id,img_no+1)
                    if os.path.isfile(this_filename):
                        image_list.append(this_filename)


        training_pairs = []
        for imagename in image_list:
            # for every image, find the corresponding view of image (positive)
            pair_group_name = imagename[-14:-12]
            identity_name = imagename[-11:-7]
            view_name = int(imagename[-6:-4])/5 
            for i in range(num_pos):
                imagename_pair = imagename[0:-6] + '%02d'%(random.randint(0,4)+1+(1-view_name)*5) + '.jpg'
                if os.path.isfile(imagename_pair):
                    pair_name = imagename + ' ' + imagename_pair + ' 1'
                    training_pairs.append(pair_name)
            # collect nagative training samples
            for i in range(num_neg):
                imagename_pair = image_list[random.randint(0,len(image_list))-1]
                identity_name_pair = imagename_pair[-11:-7]
                if identity_name_pair!=identity_name:
                    if os.path.isfile(imagename_pair):
                        pair_name = imagename + ' ' + imagename_pair + ' 0'
                        training_pairs.append(pair_name)


        random.shuffle(training_pairs)
       
        for line in training_pairs:
            fp.write(line+'\n')
            fp.flush()
        fp.close()
        print 'summary: set:%d - %s'%(set_no,phase)
        print 'person:%d'%(len(lines))
        print 'img num:%d'%len(image_list)
        print "tpairs_num:%d"%(len(training_pairs))
        

1160
summary: set:1 - train
person:1160
img num:11234
tpairs_num:807046
100
summary: set:1 - test
person:100
img num:965
tpairs_num:68893
1160
summary: set:2 - train
person:1160
img num:11236
tpairs_num:807097
100
summary: set:2 - test
person:100
img num:971
tpairs_num:69408
1160
summary: set:3 - train
person:1160
img num:11212
tpairs_num:804856
100
summary: set:3 - test
person:100
img num:975
tpairs_num:69685
1160
summary: set:4 - train
person:1160
img num:11232
tpairs_num:806725
100
summary: set:4 - test
person:100
img num:966
tpairs_num:68849
1160
summary: set:5 - train
person:1160
img num:11240
tpairs_num:807588
100
summary: set:5 - test
person:100
img num:965
tpairs_num:68843
1160
summary: set:6 - train
person:1160
img num:11222
tpairs_num:805608
100
summary: set:6 - test
person:100
img num:984
tpairs_num:70535
1160
summary: set:7 - train
person:1160
img num:11243
tpairs_num:807844
100
summary: set:7 - test
person:100
img num:959
tpairs_num:68228
1160
summary: set:8 - train
person